# Setup and Imports
Importing required libraries for data analysis, visualization, machine learning, and statistical analysis.

In [ ]:
# Corrected Cyclone Machine Data Analysis Pipeline
# Task 1: Cyclone Machine Data Analysis

import os
import sys
import json
import math
from dataclasses import dataclass
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge

#import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Set plotting parameters
plt.rcParams["figure.figsize"] = (12, 4)
sns.set_style("whitegrid")

print(" All packages imported successfully")


✓ All packages imported successfully


#  Configuration Settings
Defining configuration parameters including file paths, sampling frequency, and analysis parameters.

In [4]:
# Configuration
CONFIG = {
    "input_csv": "data.xlsx",  # Support for Excel files
    "timestamp_col": "timestamp",
    "freq_minutes": 5,
    "vars": [
        "Cyclone_Inlet_Gas_Temp",
        "Cyclone_Gas_Outlet_Temp",
        "Cyclone_Outlet_Gas_draft",
        "Cyclone_cone_draft",
        "Cyclone_Inlet_Draft",
        "Cyclone_Material_Temp",
    ],
    "plots_dir": "Task1/plots",
    "outputs_dir": "Task1/outputs",
    "outliers": {"method": "mad", "k": 5.0},
    "small_gap_limit_minutes": 30,
    "shutdown_rules": {
        "min_idle_minutes": 30,
        "max_var_window_min": 30,
        "near_zero_thresholds": {
            "Cyclone_Outlet_Gas_draft": 0.02,
            "Cyclone_cone_draft": 0.02,
        },
        "flat_std_thresholds": {
            "Cyclone_Inlet_Gas_Temp": 1.0,
            "Cyclone_Gas_Outlet_Temp": 1.0,
        }
    },
    "cluster": {
        "max_k": 6,
        "min_k": 3,
        "dbscan": {"eps": 0.8, "min_samples": 200}
    },
    "anomaly": {
        "contamination": 0.02,
        "merge_gap_minutes": 10,
        "min_event_minutes": 10
    },
    "forecast": {
        "target": "Cyclone_Inlet_Gas_Temp",
        "horizon_steps": 12,
        "train_ratio": 0.8,
        "model": "ridge"
    },
    "random_state": 42
}

# Create directories
os.makedirs(CONFIG["plots_dir"], exist_ok=True)
os.makedirs(CONFIG["outputs_dir"], exist_ok=True)

print("✓ Configuration set and directories created")
print(f"Input file: {CONFIG['input_csv']}")
print(f"Outputs: {CONFIG['outputs_dir']}")
print(f"Plots: {CONFIG['plots_dir']}")


✓ Configuration set and directories created
Input file: data.xlsx
Outputs: Task1/outputs
Plots: Task1/plots


#  Helper Functions - Basic Utilities
Utility functions for saving figures, creating directories, and basic data operations.

In [ ]:
# Basic helper functions
def savefig(path):
    """Save figure with consistent formatting"""
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches='tight')
    plt.close()

def auto_detect_timestamp_col(df):
    """Auto-detect timestamp column"""
    possible_names = ['timestamp', 'time', 'datetime', 'date', 'Timestamp', 'Time', 'DateTime']
    for col in df.columns:
        if col in possible_names or 'time' in col.lower() or 'date' in col.lower():
            return col
    # Try parsing first few rows
    for col in df.columns:
        try:
            pd.to_datetime(df[col].iloc[:5])
            return col
        except:
            continue
    return df.columns[0]

def enforce_fixed_freq(df, ts_col, freq_minutes):
    """Enforce strict time frequency"""
    df = df.copy()
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).set_index(ts_col)
    full_idx = pd.date_range(df.index.min(), df.index.max(), freq=f"{freq_minutes}min")
    df = df.reindex(full_idx)
    df.index.name = ts_col
    return df

def fill_small_gaps_ffill(df, limit_minutes, freq_minutes):
    """Forward fill small gaps"""
    limit = int(limit_minutes / freq_minutes)
    return df.ffill(limit=limit)

def winsorize_mad(series, k):
    """Robust outlier handling using MAD"""
    numeric_series = pd.to_numeric(series, errors='coerce')
    med = np.nanmedian(numeric_series)
    mad = median_abs_deviation(numeric_series, nan_policy="omit")
    if np.isnan(mad) or mad == 0:
        return numeric_series.clip(numeric_series.quantile(0.01), numeric_series.quantile(0.99))
    low = med - k * mad
    high = med + k * mad
    return numeric_series.clip(lower=low, upper=high)

print(" Basic helper functions defined")


✓ Basic helper functions defined


#  Helper Functions - Feature Engineering
Functions for creating rolling statistics, lag features, and time-based features.

In [ ]:
# Feature engineering functions
def add_rolling_features(df, cols, windows=[12, 72]):
    """Add rolling statistics and deltas"""
    df = df.copy()
    for c in cols:
        for w in windows:
            df[f"{c}_rollmean_{w}"] = df[c].rolling(w, min_periods=max(3, w//3)).mean()
            df[f"{c}_rollstd_{w}"] = df[c].rolling(w, min_periods=max(3, w//3)).std()
        df[f"{c}_delta_1"] = df[c].diff(1)
    return df

def ensure_series(x, index):
    """Ensure x is a pandas Series with given index"""
    if isinstance(x, pd.Series):
        return x.reindex(index)
    else:
        return pd.Series(x, index=index)

def build_cluster_features(df, cols):
    """Build comprehensive feature matrix for clustering"""
    X = pd.DataFrame(index=df.index)
    X = df[cols].copy()
    X = add_rolling_features(X, cols, windows=[12, 72])

    # Create lag features
    for c in cols:
        for lag in [1, 2, 12]:
            X[f"{c}_lag{lag}"] = df[c].shift(lag)

    X = X.dropna()
    return X

print("Feature engineering functions defined")


✓ Feature engineering functions defined


#  Helper Functions - Event Detection
Functions for detecting contiguous events, shutdown periods, and operational state changes.

In [ ]:
# Event detection functions
def contiguous_events(mask: pd.Series, min_len_steps: int, merge_gap_steps: int):
    """Extract contiguous events from boolean mask"""
    if not isinstance(mask, pd.Series):
        mask = pd.Series(mask, index=mask.index if hasattr(mask, 'index') else None)

    idx = mask.index
    events = []
    in_evt = False
    start = None
    last_true = None

    for t, v in zip(idx, mask.values):
        if v:
            if not in_evt:
                in_evt = True
                start = t
            last_true = t
        else:
            if in_evt:
                end = last_true
                events.append((start, end))
                in_evt = False

    if in_evt:
        events.append((start, last_true))

    # Merge nearby events
    merged = []
    for s, e in events:
        if not merged:
            merged.append([s, e])
        else:
            prev_s, prev_e = merged[-1]
            gap = (s - prev_e) / pd.Timedelta(minutes=CONFIG["freq_minutes"])
            if gap <= merge_gap_steps:
                merged[-1][1] = e
            else:
                merged.append([s, e])

    # Filter by minimum length
    filtered = []
    for s, e in merged:
        dur_steps = int(((e - s) / pd.Timedelta(minutes=CONFIG["freq_minutes"])) + 1)
        if dur_steps >= min_len_steps:
            filtered.append((s, e))
    return filtered

def detect_shutdown_mask(df, cfg):
    """Detect shutdown/idle periods using multiple criteria"""
    freq = CONFIG["freq_minutes"]
    w = max(1, int(cfg["max_var_window_min"] / freq))

    # Near-zero draft conditions
    nz_masks = []
    for col, thr in cfg["near_zero_thresholds"].items():
        if col in df:
            nz_masks.append((df[col].abs() <= thr).astype(bool))

    if nz_masks:
        nz_mask = pd.concat(nz_masks, axis=1).all(axis=1).astype(bool)
    else:
        nz_mask = pd.Series(False, index=df.index)

    # Flat temperature conditions
    flat_masks = []
    for col, thr in cfg["flat_std_thresholds"].items():
        if col in df:
            flat_masks.append((df[col].rolling(w, min_periods=max(1, w//2)).std() <= thr).astype(bool))

    if flat_masks:
        flat_mask = pd.concat(flat_masks, axis=1).any(axis=1).astype(bool)
    else:
        flat_mask = pd.Series(False, index=df.index)

    # Combine conditions
    idle = (nz_mask | flat_mask).astype(bool)
    idle = idle.reindex(df.index).fillna(False)

    # Filter by minimum duration
    min_len = max(1, int(cfg["min_idle_minutes"] / freq))
    run_ids = (idle != idle.shift(1)).cumsum()
    out = pd.Series(False, index=idle.index)
    for rid, grp in idle.groupby(run_ids):
        if grp.iloc[0] and grp.sum() >= min_len:
            out.loc[grp.index] = True
    return out

print("Event detection functions defined")


✓ Event detection functions defined


#  Data Loading and Initial Processing
Loading the dataset, performing initial validation, and data quality checks.

In [15]:
%pip install openpyxl

# Data loading and initial processing
print("Loading data...")
np.random.seed(CONFIG["random_state"])

# Load data (support both CSV and Excel)
file_ext = os.path.splitext(CONFIG["input_csv"])[1].lower()
if file_ext == ".csv":
    df_raw = pd.read_csv(CONFIG["input_csv"])
elif file_ext in [".xls", ".xlsx"]:
    df_raw = pd.read_excel(CONFIG["input_csv"])
else:
    raise ValueError(f"Unsupported file format: {file_ext}")

print(f"Raw data shape: {df_raw.shape}")
print(f"Raw columns: {list(df_raw.columns)}")

# Auto-detect timestamp column
ts_col = auto_detect_timestamp_col(df_raw)
print(f"Using timestamp column: {ts_col}")

# Check for available variables
available_vars = [v for v in CONFIG["vars"] if v in df_raw.columns]
if not available_vars:
    print("Warning: Expected variable names not found. Using all numeric columns.")
    available_vars = df_raw.select_dtypes(include=[np.number]).columns.tolist()

print(f"Available variables: {available_vars}")
df = df_raw[[ts_col] + available_vars].copy()

print("\nSample of raw data:")
display(df.head())

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.
Loading data...




[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Raw data shape: (377719, 7)
Raw columns: ['time', 'Cyclone_Inlet_Gas_Temp', 'Cyclone_Material_Temp', 'Cyclone_Outlet_Gas_draft', 'Cyclone_cone_draft', 'Cyclone_Gas_Outlet_Temp', 'Cyclone_Inlet_Draft']
Using timestamp column: time
Available variables: ['Cyclone_Inlet_Gas_Temp', 'Cyclone_Gas_Outlet_Temp', 'Cyclone_Outlet_Gas_draft', 'Cyclone_cone_draft', 'Cyclone_Inlet_Draft', 'Cyclone_Material_Temp']

Sample of raw data:


,time,Cyclone_Inlet_Gas_Temp,Cyclone_Gas_Outlet_Temp,Cyclone_Outlet_Gas_draft,Cyclone_cone_draft,Cyclone_Inlet_Draft,Cyclone_Material_Temp
0,2017-01-01 00:00:00,867.63,852.13,-189.54,-186.04,-145.9,910.42
1,2017-01-01 00:05:00,879.23,862.53,-184.33,-182.1,-149.76,918.14
2,2017-01-01 00:10:00,875.67,866.06,-181.26,-166.47,-145.01,924.18
3,2017-01-01 00:15:00,875.28,865.85,-179.15,-174.83,-142.82,923.15
4,2017-01-01 00:20:00,891.66,876.06,-178.32,-173.72,-143.39,934.26


#  Time Series Preprocessing
Enforcing fixed 5-minute intervals, handling missing timestamps, and data interpolation.

In [16]:
# Time series preprocessing
print("Enforcing fixed 5-minute frequency...")
df = enforce_fixed_freq(df, ts_col, CONFIG["freq_minutes"])

print(f"After reindexing: {df.shape}")
print(f"Missing values per column:")
missing_counts = df[available_vars].isnull().sum()
print(missing_counts)

print("\nFilling small gaps with forward fill...")
df[available_vars] = fill_small_gaps_ffill(
    df[available_vars],
    CONFIG["small_gap_limit_minutes"],
    CONFIG["freq_minutes"]
)

print("After gap filling - Missing values:")
print(df[available_vars].isnull().sum())

print("\nWinsorizing outliers using MAD method...")
for v in available_vars:
    # Ensure column is numeric, coercing errors to NaN
    df[v] = pd.to_numeric(df[v], errors='coerce')
    # Drop NaNs temporarily for range calculation if needed, or handle them
    # For robustness, calculate range only on non-NaN values
    valid_data = df[v].dropna()
    if not valid_data.empty:
        original_range = f"[{valid_data.min():.2f}, {valid_data.max():.2f}]"
        df[v] = winsorize_mad(df[v], CONFIG["outliers"]["k"])
        new_range = f"[{df[v].min():.2f}, {df[v].max():.2f}]"
        print(f"{v}: {original_range} → {new_range}")
    else:
        print(f"{v}: No valid numeric data to winsorize.")


print("✓ Data preprocessing complete")

Enforcing fixed 5-minute frequency...
After reindexing: (378580, 6)
Missing values per column:
Cyclone_Inlet_Gas_Temp      861
Cyclone_Gas_Outlet_Temp     861
Cyclone_Outlet_Gas_draft    861
Cyclone_cone_draft          861
Cyclone_Inlet_Draft         861
Cyclone_Material_Temp       861
dtype: int64

Filling small gaps with forward fill...
After gap filling - Missing values:
Cyclone_Inlet_Gas_Temp      843
Cyclone_Gas_Outlet_Temp     843
Cyclone_Outlet_Gas_draft    843
Cyclone_cone_draft          843
Cyclone_Inlet_Draft         843
Cyclone_Material_Temp       843
dtype: int64

Winsorizing outliers using MAD method...
Cyclone_Inlet_Gas_Temp: [0.00, 1157.63] → [777.18, 987.58]
Cyclone_Gas_Outlet_Temp: [13.79, 1375.00] → [700.97, 1042.07]
Cyclone_Outlet_Gas_draft: [-456.66, 40.27] → [-395.36, -35.16]
Cyclone_cone_draft: [-459.31, 488.86] → [-365.01, -32.11]
Cyclone_Inlet_Draft: [-396.37, 41.64] → [-301.56, -37.36]
Cyclone_Material_Temp: [-185.00, 1375.00] → [738.81, 1087.91]
✓ Data preproc

#  Exploratory Data Analysis
Generating summary statistics, correlation analysis, and data distribution insights.

In [17]:
# Summary statistics and correlations
print("Generating summary statistics...")

# Save summary stats
desc = df[available_vars].describe().T
desc.to_csv(os.path.join(CONFIG["outputs_dir"], "summary_stats.csv"))

print("Summary Statistics:")
display(desc)

# Correlation matrix
corr = df[available_vars].corr()
corr.to_csv(os.path.join(CONFIG["outputs_dir"], "correlations.csv"))

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, square=True)
plt.title("Variable Correlation Matrix")
savefig(os.path.join(CONFIG["plots_dir"], "correlation_matrix.png"))
plt.show()

print("✓ Summary statistics saved")


Generating summary statistics...
Summary Statistics:


,count,mean,std,min,25%,50%,75%,max
Cyclone_Inlet_Gas_Temp,376417.0,867.846982,52.617870,777.18,856.27,882.38,901.1100,987.58
Cyclone_Gas_Outlet_Temp,376416.0,841.173925,82.463520,700.97,801.97,871.52,899.3000,1042.07
Cyclone_Outlet_Gas_draft,376416.0,-185.497946,85.372483,-395.36,-247.19,-215.26,-170.1375,-35.16
Cyclone_cone_draft,376417.0,-170.719891,79.138405,-365.01,-226.77,-198.56,-143.6500,-32.11
Cyclone_Inlet_Draft,376415.0,-149.015702,63.773916,-301.56,-193.51,-169.46,-136.3000,-37.36
Cyclone_Material_Temp,376146.0,887.541680,83.687185,738.81,867.66,913.36,943.6500,1087.91


✓ Summary statistics saved


#  EDA Visualizations
Creating time series plots, correlation heatmaps, and distribution visualizations.

In [ ]:
# Create EDA visualizations
print("Creating EDA plots...")

t0 = df.index.min()
week_end = t0 + pd.Timedelta(days=7)
year_end = t0 + pd.Timedelta(days=365)

# Week slice visualization
fig, axes = plt.subplots(len(available_vars), 1, figsize=(15, 3*len(available_vars)))
if len(available_vars) == 1:
    axes = [axes]

for i, v in enumerate(available_vars):
    week_data = df.loc[t0:week_end, v]
    axes[i].plot(week_data.index, week_data.values, linewidth=0.8)
    axes[i].set_title(f"{v} (Week Slice)")
    axes[i].set_ylabel(v)
    axes[i].grid(True, alpha=0.3)

plt.suptitle("One Week Data Overview", fontsize=16)
savefig(os.path.join(CONFIG["plots_dir"], "eda_week_slice.png"))
plt.show()

# Year slice visualization
fig, axes = plt.subplots(len(available_vars), 1, figsize=(15, 3*len(available_vars)))
if len(available_vars) == 1:
    axes = [axes]

for i, v in enumerate(available_vars):
    year_data = df.loc[t0:year_end, v]
    axes[i].plot(year_data.index, year_data.values, linewidth=0.5, alpha=0.8)
    axes[i].set_title(f"{v} (Year Slice)")
    axes[i].set_ylabel(v)
    axes[i].grid(True, alpha=0.3)

plt.suptitle("One Year Data Overview", fontsize=16)
savefig(os.path.join(CONFIG["plots_dir"], "eda_year_slice.png"))
plt.show()

print(" EDA visualizations complete")


Creating EDA plots...
✓ EDA visualizations complete


#  Shutdown Detection
Identifying and analyzing machine shutdown and idle periods using multi-variable criteria.

In [20]:
# Shutdown detection
print("Detecting shutdown/idle periods...")

shutdown_mask = detect_shutdown_mask(df, CONFIG["shutdown_rules"])
min_steps = int(CONFIG["shutdown_rules"]["min_idle_minutes"] / CONFIG["freq_minutes"])
shutdown_events = contiguous_events(shutdown_mask, min_steps, merge_gap_steps=0)

# Create shutdown dataframe
shut_df = pd.DataFrame(shutdown_events, columns=["start", "end"]) if shutdown_events else pd.DataFrame(columns=["start", "end"])

if not shut_df.empty:
    shut_df["duration_min"] = ((shut_df["end"] - shut_df["start"]) / pd.Timedelta(minutes=CONFIG["freq_minutes"]) + 1) * CONFIG["freq_minutes"]
    total_downtime = shut_df["duration_min"].sum()

    print(f"✓ Found {len(shut_df)} shutdown events")
    print(f"Total downtime: {total_downtime:.0f} minutes ({total_downtime/60:.1f} hours)")
    print(f"Average shutdown duration: {shut_df['duration_min'].mean():.0f} minutes")
    print(f"Machine availability: {((len(df) - shutdown_mask.sum()) / len(df) * 100):.1f}%")

    print("\nTop 5 longest shutdowns:")
    display(shut_df.nlargest(5, 'duration_min'))
else:
    print("No shutdown events detected with current thresholds")

# Save shutdown periods
shut_df.to_csv(os.path.join(CONFIG["outputs_dir"], "shutdown_periods.csv"), index=False)
print("Shutdown periods saved")


Detecting shutdown/idle periods...
✓ Found 268 shutdown events
Total downtime: 430705 minutes (7178.4 hours)
Average shutdown duration: 1607 minutes
Machine availability: 77.2%

Top 5 longest shutdowns:


,start,end,duration_min
261,2020-03-22 21:20:00,2020-05-02 00:30:00,57795.0
217,2019-09-14 08:10:00,2019-10-18 12:20:00,49215.0
255,2019-11-29 12:20:00,2019-12-24 22:50:00,36635.0
163,2018-09-25 15:55:00,2018-10-18 10:30:00,32800.0
211,2019-06-03 03:25:00,2019-06-22 03:25:00,27365.0


Shutdown periods saved


#  Shutdown Visualization
Visualizing shutdown patterns, durations, and temporal distribution.

In [ ]:
# Shutdown visualization
print("Creating shutdown visualizations...")

# Year view with shutdown bands
plt.figure(figsize=(15, 8))
v = available_vars[0]
year_data = df.loc[t0:year_end]
plt.plot(year_data.index, year_data[v], label=v, lw=0.6, alpha=0.8)

# Add shutdown bands
shutdown_label_added = False
for s, e in shutdown_events:
    if s <= year_end:
        label = "Shutdown Periods" if not shutdown_label_added else ""
        plt.axvspan(s, min(e, year_end), color="red", alpha=0.15, label=label)
        shutdown_label_added = True

plt.legend()
plt.title("Full Year View with Shutdown Periods Highlighted")
plt.ylabel(v)
plt.grid(True, alpha=0.3)
savefig(os.path.join(CONFIG["plots_dir"], "year_with_shutdowns.png"))
plt.show()

# Shutdown frequency analysis
if not shut_df.empty:
    shut_df['month'] = pd.to_datetime(shut_df['start']).dt.month
    monthly_counts = shut_df['month'].value_counts().sort_index()

    plt.figure(figsize=(10, 4))
    monthly_counts.plot(kind='bar', color='skyblue', alpha=0.8)
    plt.title("Shutdown Events by Month")
    plt.xlabel("Month")
    plt.ylabel("Number of Events")
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3)
    savefig(os.path.join(CONFIG["plots_dir"], "shutdowns_by_month.png"))
    plt.show()

print("Shutdown visualizations complete")

Creating shutdown visualizations...
✓ Shutdown visualizations complete


#  Clustering Functions
Defining optimized K-means clustering with silhouette scoring and elbow method.

In [ ]:
# Clustering functions
def choose_kmeans_k(X_array, min_k, max_k, random_state):
    """Choose optimal K using silhouette analysis"""
    best_k, best_score = None, -1
    scores = []

    for k in range(min_k, max_k + 1):
        km = KMeans(n_clusters=k, random_state=random_state, n_init=10)
        labels = km.fit_predict(X_array)
        sc = silhouette_score(X_array, labels)
        scores.append(sc)

        if sc > best_score:
            best_k, best_score = k, sc

    # Plot silhouette scores
    plt.figure(figsize=(8, 4))
    plt.plot(range(min_k, max_k + 1), scores, 'bo-')
    plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, label=f'Best K={best_k}')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.title('K Selection via Silhouette Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    km = KMeans(n_clusters=best_k, random_state=random_state, n_init=10).fit(X_array)
    return km, best_k, best_score

def summarize_clusters(df, labels, cols):
    """Generate comprehensive cluster summary statistics"""
    dfc = df.copy()
    dfc["state"] = labels
    stats = []

    for s, grp in dfc.groupby("state"):
        row = {"state": int(s), "count": len(grp)}

        # Basic statistics
        for c in cols:
            if c in grp.columns:
                row[f"{c}_mean"] = grp[c].mean()
                row[f"{c}_std"] = grp[c].std()
                row[f"{c}_p10"] = grp[c].quantile(0.1)
                row[f"{c}_p90"] = grp[c].quantile(0.9)

        # Duration statistics
        state_mask = (dfc["state"] == s)
        state_mask = pd.Series(state_mask.values, index=dfc.index)
        state_events = contiguous_events(state_mask, 1, 0)
        if state_events:
            durations = [((e - s) / pd.Timedelta(minutes=CONFIG["freq_minutes"]) + 1) * CONFIG["freq_minutes"]
                        for s, e in state_events]
            row["avg_duration_min"] = np.mean(durations)
            row["event_count"] = len(state_events)
        else:
            row["avg_duration_min"] = 0
            row["event_count"] = 0

        stats.append(row)

    return pd.DataFrame(stats).sort_values("state")

def make_state_strip_plot(tidx, labels, path):
    """Create operational states timeline visualization"""
    dfp = pd.DataFrame({"t": tidx, "state": labels}).set_index("t")

    fig, ax = plt.subplots(figsize=(15, 4))
    scatter = ax.scatter(dfp.index, dfp["state"], s=3, c=dfp["state"],
                        cmap="tab10", marker="|", alpha=0.8)
    ax.set_ylabel("Operational State")
    ax.set_title("Machine Operational States Over Time")
    ax.grid(True, alpha=0.3)

    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label("State ID")

    savefig(path)
    plt.show()

print(" Clustering functions defined")


✓ Clustering functions defined


#  Clustering Execution
Performing operational state clustering on active (non-shutdown) data.

In [ ]:
def choose_kmeans_k_optimized(X_array, min_k, max_k, random_state, sample_size=5000):
    """
    Optimized version of choose_kmeans_k that handles large datasets more efficiently
    """
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    import numpy as np
    
    print(f"Original dataset size: {X_array.shape}")
    
    # For large datasets, use sampling for silhouette score calculation
    if len(X_array) > sample_size:
        print(f"Dataset is large, using sample of {sample_size} points for silhouette calculation")
        sample_indices = np.random.RandomState(random_state).choice(
            len(X_array), size=sample_size, replace=False
        )
        X_sample = X_array[sample_indices]
    else:
        X_sample = X_array
        sample_indices = np.arange(len(X_array))
    
    best_score = -1
    best_k = min_k
    best_km = None
    scores = []
    
    print("Testing cluster counts:")
    for k in range(min_k, max_k + 1):
        print(f"  K={k}...", end="")
        
        # Fit on full dataset but evaluate on sample
        km = KMeans(n_clusters=k, random_state=random_state, n_init=10, max_iter=300)
        km.fit(X_array)
        
        # Get labels for sample
        if len(X_array) > sample_size:
            sample_labels = km.predict(X_sample)
        else:
            sample_labels = km.labels_
        
        # Calculate silhouette score on sample
        sc = silhouette_score(X_sample, sample_labels)
        scores.append(sc)
        
        print(f" silhouette={sc:.3f}")
        
        if sc > best_score:
            best_score = sc
            best_k = k
            best_km = km
    
    return best_km, best_k, best_score


def choose_kmeans_k_elbow(X_array, min_k, max_k, random_state):
    """
    Alternative method using elbow method (inertia) which is much faster
    """
    from sklearn.cluster import KMeans
    import numpy as np
    
    inertias = []
    k_range = range(min_k, max_k + 1)
    
    print("Using elbow method for faster cluster selection:")
    for k in k_range:
        print(f"  K={k}...", end="")
        km = KMeans(n_clusters=k, random_state=random_state, n_init=10, max_iter=300)
        km.fit(X_array)
        inertias.append(km.inertia_)
        print(f" inertia={km.inertia_:.0f}")
    
    # Simple elbow detection: find the point where the rate of decrease slows down
    if len(inertias) >= 3:
        # Calculate rate of change
        rates = []
        for i in range(1, len(inertias)):
            rates.append(inertias[i-1] - inertias[i])
        
        # Find where the rate of improvement drops significantly
        rate_changes = []
        for i in range(1, len(rates)):
            rate_changes.append(rates[i-1] - rates[i])
        
        if rate_changes:
            # Pick the k where the rate change is maximum (elbow point)
            elbow_idx = np.argmax(rate_changes)
            best_k = min_k + elbow_idx + 2  # +2 because of indexing offset
        else:
            best_k = min_k + 1
    else:
        best_k = min_k + 1
    
    # Fit final model with best k
    best_km = KMeans(n_clusters=best_k, random_state=random_state, n_init=10)
    best_km.fit(X_array)
    
    return best_km, best_k, inertias[best_k - min_k]


# ============================================================================
# EXECUTE CLUSTERING ON ACTIVE DATA
# ============================================================================

print("Clustering operational states...")

# Use only active (non-shutdown) data
active = df[~shutdown_mask].copy()
print(f"Active data points: {len(active)} ({len(active)/len(df)*100:.1f}% of total)")

if len(active) < 100:
    print("Warning: Very few active data points for clustering")
else:
    # Build clustering features
    print("Building cluster features...")
    X = build_cluster_features(active[available_vars], available_vars)

    if X.empty:
        print("Warning: No features available after dropna")
    else:
        print(f"Feature matrix shape: {X.shape}")
        active_X = active.loc[X.index].copy()

        # Standardize features
        print("Standardizing features...")
        scaler = StandardScaler()
        X_fill = X.fillna(method='ffill').fillna(method='bfill')
        Xs = scaler.fit_transform(X_fill)

        # Choose optimal K - OPTIMIZED VERSION
        print("Selecting optimal number of clusters...")
        print(f"Dataset size: {Xs.shape[0]} points, {Xs.shape[1]} features")

        # Choose method based on dataset size
        if Xs.shape[0] > 10000:
            print("Large dataset detected, using elbow method for speed")
            km, best_k, best_metric = choose_kmeans_k_elbow(
                Xs,
                CONFIG['cluster']['min_k'],
                CONFIG['cluster']['max_k'],
                CONFIG['random_state']
            )
            print(f"✓ Selected {best_k} clusters with inertia: {best_metric:.0f}")
        else:
            print("Using optimized silhouette method with sampling")
            km, best_k, best_score = choose_kmeans_k_optimized(
                Xs,
                CONFIG['cluster']['min_k'],
                CONFIG['cluster']['max_k'],
                CONFIG['random_state']
            )
            print(f"✓ Selected {best_k} clusters with silhouette score: {best_score:.3f}")

        labels = km.predict(Xs)

        # Generate cluster summary
        cluster_summary = summarize_clusters(active_X.loc[X.index, available_vars], labels, available_vars)
        cluster_summary.to_csv(os.path.join(CONFIG['outputs_dir'], 'clusters_summary.csv'), index=False)

        print("\nCluster Summary:")
        display(cluster_summary[['state', 'count', 'avg_duration_min', 'event_count']].head(10))

        print("Cluster analysis complete")


Clustering operational states...
Active data points: 292439 (77.2% of total)
Building cluster features...
Feature matrix shape: (289121, 54)
Standardizing features...


C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\16422097.py:127: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_fill = X.fillna(method='ffill').fillna(method='bfill')


Selecting optimal number of clusters...
Dataset size: 289121 points, 54 features
Large dataset detected, using elbow method for speed
Using elbow method for faster cluster selection:
  K=3... inertia=11346467
  K=4... inertia=10568853
  K=5... inertia=9986639
  K=6... inertia=9573401
✓ Selected 5 clusters with inertia: 9986639

Cluster Summary:


,state,count,avg_duration_min,event_count
0,0,76661,132.354751,2926
1,1,73646,85.804293,4333
2,2,66109,77.201944,4630
3,3,62337,50.804016,6225
4,4,10368,1075.601415,424


✓ Cluster analysis complete


The code above is AI generated as the in clustering funtion below the silhouette_score function has O(n²) time complexity, making it extremely slow for large datasets . When the clustering optimization loop tries different values of k, each iteration becomes progressively slower. 

In [23]:
# Execute clustering on active data
print("Clustering operational states...")

# Use only active (non-shutdown) data
active = df[~shutdown_mask].copy()
print(f"Active data points: {len(active)} ({len(active)/len(df)*100:.1f}% of total)")

if len(active) < 100:
    print("Warning: Very few active data points for clustering")
else:
    # Build clustering features
    print("Building cluster features...")
    X = build_cluster_features(active[available_vars], available_vars)

    if X.empty:
        print("Warning: No features available after dropna")
    else:
        print(f"Feature matrix shape: {X.shape}")
        active_X = active.loc[X.index].copy()

        # Standardize features
        print("Standardizing features...")
        scaler = StandardScaler()
        X_fill = X.fillna(method="ffill").fillna(method="bfill")
        Xs = scaler.fit_transform(X_fill)

        # Choose optimal K
        print("Selecting optimal number of clusters...")
        km, best_k, best_score = choose_kmeans_k(
            Xs,
            CONFIG["cluster"]["min_k"],
            CONFIG["cluster"]["max_k"],
            CONFIG["random_state"]
        )

        labels = km.predict(Xs)
        print(f"✓ Selected {best_k} clusters with silhouette score: {best_score:.3f}")

        # Generate cluster summary
        cluster_summary = summarize_clusters(active_X.loc[X.index, available_vars], labels, available_vars)
        cluster_summary.to_csv(os.path.join(CONFIG["outputs_dir"], "clusters_summary.csv"), index=False)

        print("\nCluster Summary:")
        display(cluster_summary[['state', 'count', 'avg_duration_min', 'event_count']].head(10))

        print("✓ Cluster analysis complete")


Clustering operational states...
Active data points: 292439 (77.2% of total)
Building cluster features...
Feature matrix shape: (289121, 54)
Standardizing features...


C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\3571437509.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_fill = X.fillna(method="ffill").fillna(method="bfill")


Selecting optimal number of clusters...


KeyboardInterrupt: 

#  Clustering Visualizations
Creating state timeline plots, distribution charts, and transition matrices.

In [27]:
# Clustering visualizations
if 'labels' in locals():
    print("Creating clustering visualizations...")

    # State timeline
    make_state_strip_plot(X.index, labels,
                         os.path.join(CONFIG["plots_dir"], "operational_states_timeline.png"))

    # State distribution
    plt.figure(figsize=(8, 8))
    state_counts = pd.Series(labels).value_counts().sort_index()
    colors = plt.cm.tab10(np.arange(len(state_counts)))

    plt.pie(state_counts.values, labels=[f'State {i}' for i in state_counts.index],
            autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title("Distribution of Operational States")
    savefig(os.path.join(CONFIG["plots_dir"], "state_distribution.png"))
    plt.show()

    # State transition analysis
    state_series = pd.Series(labels, index=X.index)
    transitions = pd.DataFrame({
        'from_state': state_series.shift(1),
        'to_state': state_series
    }).dropna()

    transition_matrix = pd.crosstab(transitions['from_state'], transitions['to_state'],
                                  normalize='index') * 100

    plt.figure(figsize=(8, 6))
    sns.heatmap(transition_matrix, annot=True, fmt='.1f', cmap='Blues',
                cbar_kws={'label': 'Transition Probability (%)'})
    plt.title('State Transition Matrix (%)')
    plt.xlabel('To State')
    plt.ylabel('From State')
    savefig(os.path.join(CONFIG["plots_dir"], "state_transitions.png"))
    plt.show()

    print(" State visualizations complete")


Creating clustering visualizations...
 State visualizations complete


#  Anomaly Detection Functions
Defining Isolation Forest-based anomaly detection with configurable parameters.

In [29]:
# Anomaly detection functions
def anomalies_by_state(df_active, labels, cols, contamination, merge_gap_steps, min_event_steps):
    """State-aware anomaly detection using Isolation Forest"""
    out_rows = []
    dfc = df_active.copy()
    dfc["state"] = labels

    print("Detecting anomalies per state:")
    for s, grp in dfc.groupby("state"):
        print(f"  State {s}: {len(grp)} samples", end="")

        feats = grp[cols].copy()
        feats = feats.fillna(method="ffill").fillna(method="bfill")

        if len(feats) < 50:
            print(" (skipped - insufficient data)")
            continue

        # Fit Isolation Forest
        scaler = StandardScaler()
        Z = scaler.fit_transform(feats)

        iso = IsolationForest(
            n_estimators=200,
            contamination=contamination,
            random_state=CONFIG["random_state"],
            n_jobs=-1
        ).fit(Z)

        scores = -iso.decision_function(Z)
        grp_idx = grp.index
        mask = pd.Series(scores > np.quantile(scores, 1 - contamination), index=grp_idx)

        # Group into events
        events = contiguous_events(mask, min_event_steps, merge_gap_steps)
        print(f" -> {len(events)} anomaly events")

        for s_ts, e_ts in events:
            sub = feats.loc[s_ts:e_ts]
            z_scores = (sub - feats.mean()) / (feats.std() + 1e-6)
            implicated_vars = z_scores.abs().mean().sort_values(ascending=False).head(3).index.tolist()

            out_rows.append({
                "start": s_ts,
                "end": e_ts,
                "duration_min": int(((e_ts - s_ts) / pd.Timedelta(minutes=CONFIG["freq_minutes"])) + 1) * CONFIG["freq_minutes"],
                "state": int(s),
                "top_variables": ",".join(implicated_vars),
                "severity": float(z_scores.abs().mean().mean())
            })

    return pd.DataFrame(out_rows).sort_values("start") if out_rows else pd.DataFrame()

print("Anomaly detection functions defined")


Anomaly detection functions defined


# Anomaly Detection Execution
Detecting anomalous events across operational states and ranking by severity.

In [30]:
# Execute anomaly detection
if 'labels' in locals() and 'active_X' in locals():
    print("Detecting anomalous events...")

    contam = CONFIG["anomaly"]["contamination"]
    merge_gap_steps = int(CONFIG["anomaly"]["merge_gap_minutes"] / CONFIG["freq_minutes"])
    min_event_steps = int(CONFIG["anomaly"]["min_event_minutes"] / CONFIG["freq_minutes"])

    anomalies = anomalies_by_state(
        active_X.loc[X.index, available_vars],
        labels,
        available_vars,
        contam,
        merge_gap_steps,
        min_event_steps
    )

    if not anomalies.empty:
        anomalies.to_csv(os.path.join(CONFIG["outputs_dir"], "anomalous_periods.csv"), index=False)

        print(f"\n✓ Found {len(anomalies)} anomalous events")
        print(f"Average anomaly duration: {anomalies['duration_min'].mean():.1f} minutes")

        # Anomalies by state
        if 'state' in anomalies.columns:
            state_anomaly_counts = anomalies['state'].value_counts().sort_index()
            print(f"Anomalies by state:")
            for state, count in state_anomaly_counts.items():
                print(f"  State {state}: {count} events")

        print("\nTop 5 most severe anomalies:")
        top_anomalies = anomalies.nlargest(5, 'severity')
        display(top_anomalies[['start', 'end', 'duration_min', 'state', 'severity', 'top_variables']])

    else:
        print("No anomalies detected")
        pd.DataFrame().to_csv(os.path.join(CONFIG["outputs_dir"], "anomalous_periods.csv"), index=False)

else:
    print("Skipping anomaly detection - clustering not completed")
    pd.DataFrame().to_csv(os.path.join(CONFIG["outputs_dir"], "anomalous_periods.csv"), index=False)


Detecting anomalous events...
Detecting anomalies per state:
  State 0: 76661 samples

C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\3014566497.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill")


 -> 253 anomaly events
  State 1: 73646 samples

C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\3014566497.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill")


 -> 197 anomaly events
  State 2: 66109 samples

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\3014566497.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill")


 -> 203 anomaly events
  State 4: 10368 samples

C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\3014566497.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill")


 -> 43 anomaly events

✓ Found 904 anomalous events
Average anomaly duration: 117.0 minutes
Anomalies by state:
  State 0: 253 events
  State 1: 197 events
  State 2: 208 events
  State 3: 203 events
  State 4: 43 events

Top 5 most severe anomalies:


,start,end,duration_min,state,severity,top_variables
245,2020-07-18 13:00:00,2020-07-18 13:05:00,10,0,4.395200,"Cyclone_Inlet_Draft,Cyclone_Outlet_Gas_draft,C..."
120,2018-08-01 00:00:00,2018-08-01 00:15:00,20,0,4.343986,"Cyclone_Outlet_Gas_draft,Cyclone_Inlet_Draft,C..."
246,2020-07-19 04:45:00,2020-07-19 04:50:00,10,0,4.126002,"Cyclone_Inlet_Draft,Cyclone_Outlet_Gas_draft,C..."
330,2017-12-12 11:00:00,2017-12-12 11:05:00,10,1,3.853583,"Cyclone_Inlet_Gas_Temp,Cyclone_Inlet_Draft,Cyc..."
51,2018-06-14 15:20:00,2018-06-14 15:25:00,10,0,3.850898,"Cyclone_Outlet_Gas_draft,Cyclone_Inlet_Draft,C..."


#  Anomaly Visualizations
Visualizing anomaly timeline, severity distribution, and implicated variables.

In [31]:
# Anomaly visualizations
if 'anomalies' in locals() and not anomalies.empty:
    print("Creating anomaly visualizations...")

    # Select top anomalies for detailed plots
    top_anomalies = anomalies.nlargest(min(5, len(anomalies)), 'severity')

    for i, (idx, row) in enumerate(top_anomalies.iterrows()):
        s, e = row["start"], row["end"]
        state = row["state"]

        # Context window (2 hours before/after)
        context_start = s - pd.Timedelta(hours=2)
        context_end = e + pd.Timedelta(hours=2)
        rng = slice(context_start, context_end)

        fig, axes = plt.subplots(len(available_vars), 1,
                               figsize=(15, 2.5*len(available_vars)))
        if len(available_vars) == 1:
            axes = [axes]

        for j, v in enumerate(available_vars):
            data_slice = df.loc[rng, v].dropna()
            if not data_slice.empty:
                # Z-score normalization
                normalized = (data_slice - df[v].mean()) / df[v].std()
                axes[j].plot(data_slice.index, normalized, label=v, lw=0.8, alpha=0.8)

                # Highlight anomaly period
                axes[j].axvspan(s, e, color="red", alpha=0.3,
                               label="Anomaly Period" if j == 0 else "")

                # Reference lines
                axes[j].axhline(y=2, color='orange', linestyle='--', alpha=0.5)
                axes[j].axhline(y=-2, color='orange', linestyle='--', alpha=0.5)

                axes[j].set_ylabel(f"{v}\n(z-score)")
                axes[j].grid(True, alpha=0.3)
                axes[j].legend()

        plt.suptitle(f"Anomaly Event {i+1} (State {state})\n"
                    f"{s.strftime('%Y-%m-%d %H:%M')} to {e.strftime('%Y-%m-%d %H:%M')}\n"
                    f"Severity: {row['severity']:.3f}, Variables: {row['top_variables']}",
                    fontsize=12)

        savefig(os.path.join(CONFIG["plots_dir"], f"anomaly_event_{i+1}.png"))
        plt.show()

    # Anomaly frequency over time
    plt.figure(figsize=(12, 4))
    anomaly_dates = pd.to_datetime(anomalies['start']).dt.date
    anomaly_counts = pd.Series(anomaly_dates).value_counts().sort_index()

    plt.plot(anomaly_counts.index, anomaly_counts.values, 'ro-', alpha=0.7)
    plt.title('Anomaly Events Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Events')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    savefig(os.path.join(CONFIG["plots_dir"], "anomaly_frequency.png"))
    plt.show()

    print("✓ Anomaly visualizations complete")
else:
    print("No anomalies to visualize")


Creating anomaly visualizations...
✓ Anomaly visualizations complete


#  Forecasting Functions
Defining Ridge regression-based time series forecasting pipeline.

In [34]:
# Forecasting functions
def forecasting_pipeline(df, target, horizon, model_type):
    """Complete forecasting pipeline with model comparison"""

    print(f"Building forecasting model for {target}...")
    y = df[target].astype(float)

    # Create feature matrix
    X = pd.DataFrame(index=y.index)
    for lag in [1, 2, 3, 6, 12, 24]:
        X[f"lag_{lag}"] = y.shift(lag)

    # Rolling features
    X["rollmean_12"] = y.rolling(12, min_periods=6).mean()
    X["rollstd_12"] = y.rolling(12, min_periods=6).std()

    # Combine and clean
    data = pd.concat([y, X], axis=1).dropna()
    print(f"Feature matrix shape: {data.shape}")

    y_clean = data[target]
    X_clean = data.drop(columns=[target])

    # Train/test split
    split_idx = int(len(data) * CONFIG["forecast"]["train_ratio"])
    X_train, X_test = X_clean.iloc[:split_idx], X_clean.iloc[split_idx:]
    y_train, y_test = y_clean.iloc[:split_idx], y_clean.iloc[split_idx:]

    print(f"Train: {len(y_train)}, Test: {len(y_test)} samples")

    # Baseline: Persistence
    y_pred_baseline = y_test.shift(1).fillna(y_test.iloc[0])

    # Advanced model - Ridge regression only
    print("Fitting Ridge regression...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.fillna(method='ffill'))
    X_test_scaled = scaler.transform(X_test.fillna(method='ffill'))

    ridge = Ridge(alpha=1.0, random_state=CONFIG["random_state"])
    ridge.fit(X_train_scaled, y_train)

    y_pred_model = pd.Series(ridge.predict(X_test_scaled), index=y_test.index)
    print("✓ Ridge fitted successfully")

    # Calculate metrics
    def rmse(actual, predicted):
        return float(np.sqrt(np.mean((np.array(actual) - np.array(predicted))**2)))

    def mae(actual, predicted):
        return float(np.mean(np.abs(np.array(actual) - np.array(predicted))))

    metrics = {
        "model_type": "ridge",
        "rmse_model": rmse(y_test, y_pred_model),
        "mae_model": mae(y_test, y_pred_model),
        "rmse_baseline": rmse(y_test, y_pred_baseline),
        "mae_baseline": mae(y_test, y_pred_baseline),
        "improvement_rmse": ((rmse(y_test, y_pred_baseline) - rmse(y_test, y_pred_model)) / rmse(y_test, y_pred_baseline)) * 100,
        "improvement_mae": ((mae(y_test, y_pred_baseline) - mae(y_test, y_pred_model)) / mae(y_test, y_pred_baseline)) * 100
    }

    print(f"\nForecasting Results:")
    print(f"Model RMSE: {metrics['rmse_model']:.3f}")
    print(f"Baseline RMSE: {metrics['rmse_baseline']:.3f}")
    print(f"RMSE Improvement: {metrics['improvement_rmse']:.1f}%")

    # Save results
    with open(os.path.join(CONFIG["outputs_dir"], "forecast_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    forecasts_df = pd.DataFrame({
        "timestamp": y_test.index,
        "y_true": y_test.values,
        "y_pred_baseline": y_pred_baseline.values,
        "y_pred_model": y_pred_model.values,
    })
    forecasts_df.to_csv(os.path.join(CONFIG["outputs_dir"], "forecasts.csv"), index=False)

    return y_test, y_pred_baseline, y_pred_model, metrics


print(" Forecasting functions defined")


 Forecasting functions defined


Forecasting Execution

In [35]:
# Execute forecasting
print("Running forecasting...")

# Select target variable
target_var = CONFIG["forecast"]["target"] if CONFIG["forecast"]["target"] in available_vars else available_vars[0]
print(f"Forecasting target: {target_var}")

# Prepare clean data (mask shutdowns)
clean_for_forecast = df[[target_var]].copy()

if 'shutdown_mask' in locals():
    print("Masking shutdown periods...")
    clean_for_forecast.loc[shutdown_mask, target_var] = np.nan
    print(f"Masked {shutdown_mask.sum()} shutdown points")

# Interpolate gaps
clean_for_forecast[target_var] = clean_for_forecast[target_var].interpolate(limit_direction="both")

print(f"Data for forecasting: {len(clean_for_forecast)} points")
print(f"Missing values: {clean_for_forecast[target_var].isnull().sum()}")

# Run forecasting
y_true, y_baseline, y_model, forecast_metrics = forecasting_pipeline(
    clean_for_forecast,
    target_var,
    CONFIG["forecast"]["horizon_steps"],
    CONFIG["forecast"]["model"]
)

print("✓ Forecasting complete")


Running forecasting...
Forecasting target: Cyclone_Inlet_Gas_Temp
Masking shutdown periods...
Masked 86141 shutdown points
Data for forecasting: 378580 points
Missing values: 0
Building forecasting model for Cyclone_Inlet_Gas_Temp...
Feature matrix shape: (378556, 9)
Train: 302844, Test: 75712 samples
Fitting Ridge regression...


C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\2263806623.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train_scaled = scaler.fit_transform(X_train.fillna(method='ffill'))
C:\Users\Admin\AppData\Local\Temp\ipykernel_24972\2263806623.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test_scaled = scaler.transform(X_test.fillna(method='ffill'))


✓ Ridge fitted successfully

Forecasting Results:
Model RMSE: 12.267
Baseline RMSE: 16.396
RMSE Improvement: 25.2%
✓ Forecasting complete


Forecasting Visualization


In [37]:
# Forecasting visualizations
if 'y_true' in locals():
    print("Creating forecasting visualizations...")

    # Main comparison plot
    sample_size = min(1000, len(y_true))
    sample_slice = slice(0, sample_size)

    plt.figure(figsize=(15, 8))

    plt.plot(y_true.iloc[sample_slice].index, y_true.iloc[sample_slice].values,
             label="True Values", lw=1.5, alpha=0.8, color='black')

    plt.plot(y_baseline.iloc[sample_slice].index, y_baseline.iloc[sample_slice].values,
             label=f"Baseline (RMSE: {forecast_metrics['rmse_baseline']:.3f})",
             alpha=0.7, linestyle='--', color='orange')

    plt.plot(y_model.iloc[sample_slice].index, y_model.iloc[sample_slice].values,
             label=f"Model (RMSE: {forecast_metrics['rmse_model']:.3f})",
             alpha=0.8, color='red')

    plt.legend(fontsize=12)
    plt.title(f"Forecasting Comparison - {target_var}\n"
              f"RMSE Improvement: {forecast_metrics['improvement_rmse']:.1f}%, "
              f"MAE Improvement: {forecast_metrics['improvement_mae']:.1f}%",
              fontsize=14)
    plt.ylabel(target_var)
    plt.grid(True, alpha=0.3)

    savefig(os.path.join(CONFIG["plots_dir"], "forecast_comparison.png"))
    plt.show()

    # Error analysis
    baseline_errors = y_true - y_baseline
    model_errors = y_true - y_model

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Error distributions
    axes[0].hist(baseline_errors.dropna(), bins=50, alpha=0.7, label='Baseline', density=True)
    axes[0].hist(model_errors.dropna(), bins=50, alpha=0.7, label='Model', density=True)
    axes[0].set_xlabel('Prediction Error')
    axes[0].set_ylabel('Density')
    axes[0].set_title('Error Distribution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Errors over time
    sample_errors = slice(0, min(500, len(model_errors)))
    axes[1].plot(model_errors.iloc[sample_errors].index,
                model_errors.iloc[sample_errors].values,
                alpha=0.8, color='red', lw=0.8)
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Model Error')
    axes[1].set_title('Model Errors Over Time')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    savefig(os.path.join(CONFIG["plots_dir"], "forecast_error_analysis.png"))
    plt.show()

    print(" Forecasting visualizations complete")


Creating forecasting visualizations...
 Forecasting visualizations complete


Insights Generation and Final Summary

In [39]:
# Generate insights and final summary
print("Generating insights and recommendations...")

insights = []
recommendations = []

# Data quality insights
total_points = len(df)
missing_before = df_raw[available_vars].isnull().sum().sum()
insights.append(f"Processed {total_points:,} data points over {(df.index.max() - df.index.min()).days} days")
if missing_before > 0:
    insights.append(f"Handled {missing_before:,} missing values using forward-fill and interpolation")

# Shutdown insights
if 'shut_df' in locals() and not shut_df.empty:
    total_downtime_hours = shut_df['duration_min'].sum() / 60
    availability = ((len(df) - shutdown_mask.sum()) / len(df)) * 100

    insights.append(f"Machine availability: {availability:.1f}% ({len(shut_df)} shutdown events)")
    insights.append(f"Total downtime: {total_downtime_hours:.1f} hours, avg shutdown: {shut_df['duration_min'].mean():.0f} min")

    recommendations.extend([
        "Implement predictive maintenance to reduce unplanned shutdowns",
        "Set up automated alerts for shutdowns exceeding 60 minutes",
        "Analyze shutdown patterns for seasonal maintenance planning"
    ])

# Clustering insights
if 'cluster_summary' in locals() and not cluster_summary.empty:
    n_states = len(cluster_summary)
    dominant_state = cluster_summary.loc[cluster_summary['count'].idxmax(), 'state']
    dominant_pct = (cluster_summary.loc[cluster_summary['count'].idxmax(), 'count'] /
                   cluster_summary['count'].sum()) * 100

    insights.append(f"Identified {n_states} operational states, State {dominant_state} dominant ({dominant_pct:.1f}%)")

    recommendations.extend([
        "Monitor state transitions for early degradation detection",
        "Optimize operations to maximize time in efficient states",
        "Investigate short-duration states for stability improvements"
    ])

# Anomaly insights
if 'anomalies' in locals() and not anomalies.empty:
    anomaly_rate = len(anomalies) / (len(df) / (24 * 60 / CONFIG["freq_minutes"]))

    insights.append(f"Detected {len(anomalies)} anomalous events (rate: {anomaly_rate:.2f} events/day)")

    if 'state' in anomalies.columns:
        most_anomalous_state = anomalies['state'].mode().iloc[0]
        state_count = (anomalies['state'] == most_anomalous_state).sum()
        insights.append(f"State {most_anomalous_state} shows highest anomaly frequency ({state_count} events)")

    recommendations.extend([
        "Implement real-time anomaly monitoring with severity thresholds",
        "Focus maintenance on high-anomaly operational states",
        "Investigate top implicated variables for root cause analysis"
    ])

# Forecasting insights
if 'forecast_metrics' in locals():
    improvement = forecast_metrics.get('improvement_rmse', 0)
    if improvement > 10:
        insights.append(f"Forecasting model shows good performance ({improvement:.1f}% RMSE improvement)")
    elif improvement > 0:
        insights.append(f"Forecasting shows modest improvement ({improvement:.1f}% RMSE reduction)")
    else:
        insights.append("Forecasting challenging due to high variability")

    recommendations.extend([
        "Use 1-hour forecasts for operational planning",
        "Consider state-aware forecasting for better accuracy",
        "Implement forecasting in maintenance scheduling"
    ])

# Save comprehensive insights - FIXED: Added encoding='utf-8' to handle special characters
insight_text = f"""
CYCLONE MACHINE DATA ANALYSIS - COMPREHENSIVE INSIGHTS
{'='*70}

SUMMARY INSIGHTS:
{chr(10).join([f"{i+1}. {insight}" for i, insight in enumerate(insights)])}

ACTIONABLE RECOMMENDATIONS:
{chr(10).join([f"- {rec}" for rec in recommendations])}

TECHNICAL SUMMARY:
- Dataset: {(df.index.max() - df.index.min()).days} days, {len(df):,} points at 5-min intervals
- Variables: {len(available_vars)} sensor measurements
- Machine availability: {((len(df) - shutdown_mask.sum()) / len(df) * 100):.1f}%
- Operational states: {best_k if 'best_k' in locals() else 'N/A'} distinct patterns
- Anomaly detection: {len(anomalies) if 'anomalies' in locals() and not anomalies.empty else 0} events
- Forecasting improvement: {forecast_metrics.get('improvement_rmse', 0):.1f}% RMSE reduction

DELIVERABLES GENERATED:
[DONE] shutdown_periods.csv - {len(shut_df) if not shut_df.empty else 0} shutdown events
[DONE] clusters_summary.csv - {best_k if 'best_k' in locals() else 0} operational state profiles
[DONE] anomalous_periods.csv - {len(anomalies) if 'anomalies' in locals() and not anomalies.empty else 0} anomaly events
[DONE] forecasts.csv - Model vs baseline predictions
[DONE] Comprehensive visualizations and analysis plots

NEXT STEPS:
1. Validate findings with operations team
2. Implement real-time monitoring dashboard
3. Set up automated alerting system
4. Plan predictive maintenance program
5. Consider additional sensor integration
"""

# FIXED: Added encoding='utf-8' to handle Unicode characters
with open(os.path.join(CONFIG["outputs_dir"], "insights_summary.txt"), "w", encoding='utf-8') as f:
    f.write(insight_text)

print("="*60)
print("TASK 1 ANALYSIS COMPLETE")
print("="*60)

print("\nKey Insights:")
for i, insight in enumerate(insights[:5], 1):
    print(f"{i}. {insight}")

print(f"\n[DONE] All results saved to: {CONFIG['outputs_dir']}")
print(f"[DONE] All plots saved to: {CONFIG['plots_dir']}")

# Verify deliverables
required_files = ["shutdown_periods.csv", "anomalous_periods.csv", "clusters_summary.csv", "forecasts.csv"]
print(f"\nRequired deliverables:")
for file in required_files:
    path = os.path.join(CONFIG["outputs_dir"], file)
    status = "[DONE]" if os.path.exists(path) else "[MISSING]"
    size = f"({os.path.getsize(path)} bytes)" if os.path.exists(path) else ""
    print(f"{status} {file} {size}")




Generating insights and recommendations...
TASK 1 ANALYSIS COMPLETE

Key Insights:
1. Processed 378,580 data points over 1314 days
2. Machine availability: 77.2% (268 shutdown events)
3. Total downtime: 7178.4 hours, avg shutdown: 1607 min
4. Identified 5 operational states, State 0 dominant (26.5%)
5. Detected 904 anomalous events (rate: 0.69 events/day)

[DONE] All results saved to: Task1/outputs
[DONE] All plots saved to: Task1/plots

Required deliverables:
[DONE] shutdown_periods.csv (12530 bytes)
[DONE] anomalous_periods.csv (120720 bytes)
[DONE] clusters_summary.csv (2466 bytes)
[DONE] forecasts.csv (4427799 bytes)
